In [3]:
!pip install -q --upgrade pip
!pip install -U jaxlib==0.4.4+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 83.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.


In [4]:
# https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
#%env JAX_PLATFORMS=cpu
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform

env: XLA_PYTHON_CLIENT_PREALLOCATE=false
env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [6]:
from utils import *
from layers import two_wl_aggregation

In [7]:
import numpy
import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax
import shutil
import os

In [8]:
# for input "kernel_fn(x1, x2, 'ntk', pattern = (p1,p2))"
# the shape of the kernel after the convolution is:
# x1 = (b1, h1, w1, c1)
# x2 = (b2, h2, w2, c2)
# k = (b1, b2, h1, h2, w1, w2) 


L_branche = stax.serial(
    stax.Conv(100, (1,1)),
    )

Gamma_branche = stax.serial(
    stax.Conv(100, (1,1)),
    two_wl_aggregation(n_nodes=28),
)

two_wl_aggregation_layer = stax.serial(
    stax.FanOut(2), 
    stax.parallel(L_branche, Gamma_branche),
    stax.FanInSum(),
    stax.Relu(),
    )


init_fn, apply_fn, kernel_fn = stax.serial(
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    two_wl_aggregation_layer,
    # two_wl_aggregation_layer,
    # two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer,
    #two_wl_aggregation_layer, # k = (b1, b2, h1, h2, w1, w2)
    stax.GlobalSumPool(), # # k = (b1, b2)
    stax.Dense(1),
)

In [9]:
# load preprocessed data

twl_radius = 4 
dataset_name = "MUTAG"
base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/{dataset_name}"

ys = np.load(base_path + f"/ys.npy")
graphs_edge_features = np.load(base_path + f"/graphs_edge_features.npy")
twl_pattern = np.load(base_path + f"/two_wl_pattern_radius_{twl_radius}.npy")

print(graphs_edge_features.shape) # (188, 28, 1, 7)
print(twl_pattern.shape) # (188, 66, 1, 2)

(188, 28, 28, 8)
(1504, 4)


In [ ]:
188 / 15

12.533333333333333

In [13]:
# calculate the kernel range wise 
size = 15
dataset_size = 188
r = 12

base_path = f"/content/drive/MyDrive/MasterarbeitData"
data_path = base_path + f"/Preprocessed/MUTAG"
split_data_path = data_path + f"/splits/size_{size}_range_{r}" 
kernel_path = base_path + f"/Kernels/MUTAG/TWO_WL/radius_{twl_radius}"

# clear the kernel folder
# shutil.rmtree(kernel_path)

if not os.path.exists(kernel_path):
    os.makedirs(kernel_path)


for i in range(r+1):
  for j in range(r+1):

    l = lambda x : x*size 
    h = lambda x : (1+x)*size if (not x == r) else dataset_size-1
    
    x1_batch_range = list(range(l(i),h(i)))
    x2_batch_range = list(range(l(j),h(j)))
    
    x1 = graphs_edge_features[x1_batch_range,:]
    x2 = graphs_edge_features[x2_batch_range,:]
      
    p1 = twl_pattern[column_in_values(twl_pattern[:,0], np.array(x1_batch_range)),:]
    p2 = twl_pattern[column_in_values(twl_pattern[:,0], np.array(x2_batch_range)),:]

    kernel_matrix = kernel_fn(x1, x2, 'ntk', pattern=(p1, p2))
    np.save(kernel_path + f"/NTK_{h(i)}_{h(j)}", kernel_matrix)

/usr/local/lib/python3.8/dist-packages/neural_tangents/_src/stax/branching.py:399: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'
/usr/local/lib/python3.8/dist-packages/neural_tangents/_src/stax/branching.py:399: UserWarning: `FanIn` layers assume independent inputs which is not verified in the code. Please make sure to have at least one `Dense` / `Conv` / `GlobalSelfAttention` etc. layer in each branch.
  warnings.warn('`FanIn` layers assume independent inputs which is not verified'


In [ ]:
#this needs ~9GB gpu-ram, takes ~15s on GPU ~2min on CPU (size 4 using XLA_PYTHON_CLIENT_PREALLOCATE=false)
%%time 
size = 10
r = 18
i = 0
j = 0

path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/MUTAG/splits/size_{size}_range_{r}"

x1 = np.load(path + f"/graphs_edge_features_1_{i*size}_{(1+i)*size}.npy")
x2 = np.load(path + f"/graphs_edge_features_2_{j*size}_{(1+j)*size}.npy")
p1 = np.load(path + f"/two_wl_pattern_radius_1_1_{i*size}_{(1+i)*size}.npy")
p2 = np.load(path + f"/two_wl_pattern_radius_1_2_{j*size}_{(1+j)*size}.npy")

kernel_matrix = kernel_fn(x1, x2, 'ntk', pattern=(p1, p2))

In [ ]:
dataset_size = 188
size = 3
r = 62

base_path = f"/content/drive/MyDrive/MasterarbeitData"
data_path = base_path + f"/Preprocessed/MUTAG"
split_data_path = data_path + f"/splits/size_{size}_range_{r}" 
kernel_path = base_path + f"/Kernels/MUTAG/TWO_WL"
if not os.path.exists(kernel_path):
    os.makedirs(kernel_path)

In [ ]:
# clear the kernel folder
# shutil.rmtree(kernel_path)

In [ ]:
# at some point this allways fails with 15gb gpu memory
# fails always for some function in utils
# can we make those function more memory efficient?

# with XLA_PYTHON_CLIENT_ALLOCATOR=platform this seems to stay below 5GB 
# GPU-RAM usage for size=4

# calculate kernel from saved preprocessed data
for i in range(r+1):
  for j in range(r+1):

    l = lambda x : x*size 
    h = lambda x : (1+x)*size if (not x == r) else dataset_size-1

    x1 = np.load(split_data_path + f"/graphs_edge_features_1_{l(i)}_{h(i)}.npy")
    x2 = np.load(split_data_path + f"/graphs_edge_features_2_{l(j)}_{h(j)}.npy")
    p1 = np.load(split_data_path + f"/two_wl_pattern_radius_1_1_{l(i)}_{h(i)}.npy")
    p2 = np.load(split_data_path + f"/two_wl_pattern_radius_1_2_{l(j)}_{h(j)}.npy")

    kernel_matrix = kernel_fn(x1, x2, 'ntk', pattern=(p1, p2))
    np.save(kernel_path + f"/NTK_{h(i)}_{h(j)}", kernel_matrix)

Calculate the kernel 1x1 

In [ ]:
# load preprocessed data

dataset_name = "MUTAG"
base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/{dataset_name}"

ys = np.load(base_path + f"/ys.npy")
graphs_edge_features = np.load(base_path + f"/graphs_edge_features.npy")
graph_two_wl_pattern = np.load(base_path + f"/two_wl_pattern_radius_1.npy")

print(graphs_edge_features.shape)
print(graph_two_wl_pattern.shape)

In [ ]:
i = 3
j = 100
tx1 = np.expand_dims(graphs_edge_features[i,:], 0)
tx2 = np.expand_dims(graphs_edge_features[j,:], 0)
tp1_ind = column_in_values(graph_two_wl_pattern[:,0], np.array([i]))
tp2_ind = column_in_values(graph_two_wl_pattern[:,0], np.array([j]))
tp1 = graph_two_wl_pattern[tp1_ind,:]  
tp2 = graph_two_wl_pattern[tp2_ind,:]  

kernel_fn(tx1, tx2, 'ntk', pattern=(tp1, tp2))

In [ ]:
i = 3
j = 3
tx1 = np.expand_dims(graphs_edge_features[i,:], 0)
tx2 = np.expand_dims(graphs_edge_features[j,:], 0)
tp1_ind = column_in_values(graph_two_wl_pattern[:,0], np.array([i]))
tp2_ind = column_in_values(graph_two_wl_pattern[:,0], np.array([j]))
tp1 = graph_two_wl_pattern[tp1_ind,:]  
tp2 = graph_two_wl_pattern[tp2_ind,:]  

kernel_fn(tx1, tx2, 'ntk', pattern=(tp1, tp2))

In [ ]:
kernel_matrix = np.empty((188, 188))

In [ ]:
len_x1 = 188
len_x2 = 188

for i in range(len_x1):
  x1 = graphs_edge_features[i,:]
  x1 = np.expand_dims(x1, 0)
  x1_batch_range = list(range(i*size,(1+i)*size))
  p1 = graph_two_wl_pattern[column_in_values(graph_two_wl_pattern[:,0], np.array(x1_batch_range)),:]  
  
  for j in range(len_x2):
    x2 = graphs_edge_features[j,:]
    x2 = np.expand_dims(x2, 0)
    x2_batch_range = list(range(j*size,(1+j)*size))
    p2 = graph_two_wl_pattern[column_in_values(graph_two_wl_pattern[:,0], np.array(x2_batch_range)),:]  

    k = kernel_fn(x1, x2, 'ntk', pattern=(p1, p2))
    kernel_matrix = kernel_matrix.at[i,j].set(np.squeeze(k)) 
    
np.save(kernel_path + f"/NTK_0_188_0_188", kernel_matrix)

In [ ]:
np.squeeze(k)